In [1]:
import tensorflow as tf
import numpy as np
import tensornets as nets
import cv2
from PIL import Image, ImageOps
import glob


In [2]:
def normalize(x):
    _max=np.max(x)
    _min=np.min(x)
    return (x-_min)/(_max-_min)

def load_img(path):
    image_list=[]
    for filename in glob.glob(path): 
        im=Image.open(filename)
        image_list.append(normalize(np.array(ImageOps.fit(im ,(224,224), Image.ANTIALIAS))))

    return image_list


tf.reset_default_graph()

In [3]:
batch_1=load_img('Traindata/leukocoria/*.jpg')
batch_2=load_img('Traindata/non-leukocoria/*.jpg')
test_batch1=load_img('testdata/leuko/*.jpg')
test_batch2=load_img('testdata/non-leuko/*.jpg')

In [4]:
x=tf.placeholder(dtype=tf.float32, shape=[None, 224,224, 3], name='features')
y=tf.placeholder(dtype=tf.float32, shape=[None,2], name='labels')
is_train=tf.placeholder_with_default(False, shape=())

In [5]:
epochs=20
batch_size=80
display_step=1

In [6]:
logits=nets.DenseNet169(x, is_training=is_train, classes=2)
train_weights=logits.get_weights()
loss=tf.losses.softmax_cross_entropy(y, logits)
#train=tf.train.AdamOptimizer(1e-3).minimize(loss) 
update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):               
    train=tf.train.AdamOptimizer(1e-5).minimize(loss, var_list=train_weights[600:]) 
correctPred=tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
acc=tf.reduce_mean(tf.cast(correctPred, tf.float32))

init=tf.global_variables_initializer()
local_init_op=tf.local_variables_initializer()

saver=tf.train.Saver()
save_file='./logs.ckpt'

In [7]:
len(train_weights)

846

In [8]:
logits.print_summary()

Scope: densenet169
Total layers: 169
Total weights: 1142
Total parameters: 22,328,646


In [9]:
def get_batches(batch_size, batch_1, batch_2):
    out_batches=[]
    for start in range(0, len(batch_1), batch_size):
        end=start+batch_size//2
        features=batch_1[start:end]+batch_2[start:end]
        labels=[[0.,1.]*(batch_size//2)]+[[1.,0.,]*(batch_size//2)]
        batches=[np.array(features), np.reshape(labels, (batch_size,2))]
        out_batches.append(batches)
    return out_batches

valid_feat=batch_2[-65:]+batch_1[-65:]
valid_features_=np.array(valid_feat)
valid_lab=[[1.,0.]*(65)]+[[0.,1.]*(65)]
valid_labels_=np.reshape(valid_lab,(130,2))

test_feat=test_batch2+test_batch1
test_features_=np.array(test_feat)
test_lab=[[1.,0.]*(30)]+[[0.,1.]*(30)]
test_labels_=np.reshape(test_lab,(60,2))

def epoch_stats(sess, epoch, last_feat, last_lab, valid_features,valid_labels):
    cur_cost=sess.run(loss, feed_dict={x:last_feat, y:last_lab, is_train:True})
    cur_Acc=sess.run(acc, feed_dict={x:valid_features, y:valid_labels, is_train:True})
    print('epoch : {:<4} - cost : {:<8.3} - acc: {:<5.3}'.format(epoch, cur_cost, cur_Acc) )

In [10]:
from random import shuffle
with tf.Session() as sess:
    sess.run(init)
    sess.run(local_init_op)
    sess.run(logits.pretrained())
    for e in range(epochs):
        for train_feat, train_lab in get_batches(batch_size, batch_1, batch_2):
            ind_list = [i for i in range(len(train_feat))]
            shuffle(ind_list)
            train_new  = train_feat[ind_list, :,:,:]
            target_new = train_lab[ind_list,]
            sess.run(train, feed_dict={x:train_new, y:target_new, is_train:True})
        if(e%display_step==0):
            epoch_stats(sess, e, train_new, target_new, valid_features_,valid_labels_)
            
            
    model_acc=sess.run(acc, feed_dict={x:test_features_, y:test_labels_, is_train:True})
    print(model_acc)        
    saver.save(sess,save_file)           
        
        

epoch : 0    - cost : 0.811    - acc: 0.692
epoch : 1    - cost : 0.802    - acc: 0.785
epoch : 2    - cost : 0.789    - acc: 0.815
epoch : 3    - cost : 0.771    - acc: 0.869
epoch : 4    - cost : 0.748    - acc: 0.915
epoch : 5    - cost : 0.722    - acc: 0.915
epoch : 6    - cost : 0.694    - acc: 0.923
epoch : 7    - cost : 0.665    - acc: 0.946
epoch : 8    - cost : 0.638    - acc: 0.954
epoch : 9    - cost : 0.612    - acc: 0.954
epoch : 10   - cost : 0.585    - acc: 0.946
epoch : 11   - cost : 0.56     - acc: 0.946
epoch : 12   - cost : 0.538    - acc: 0.946
epoch : 13   - cost : 0.519    - acc: 0.954
epoch : 14   - cost : 0.505    - acc: 0.954
epoch : 15   - cost : 0.494    - acc: 0.954
epoch : 16   - cost : 0.485    - acc: 0.954
epoch : 17   - cost : 0.478    - acc: 0.946
epoch : 18   - cost : 0.472    - acc: 0.946
epoch : 19   - cost : 0.468    - acc: 0.938
0.95


In [11]:
evaluate=load_img('Evaluation data/*.jpg')

In [ ]:
samp1=batch_1
samp2=batch_2

In [12]:

with tf.Session() as sess:
    loader = tf.train.import_meta_graph('./logs.ckpt.meta')
    loader.restore(sess, save_file) 
    predict1=[]
    #predict2=[]
    for start in range(0,len(evaluate), 100):
        end=start+100
        predict1.extend(sess.run(tf.argmax(logits,1), feed_dict={x:evaluate[start:end]}))
        #predict2.extend(sess.run(tf.argmax(logits,1), feed_dict={x:samp2[start:end]}))
        


INFO:tensorflow:Restoring parameters from ./logs.ckpt


In [13]:
from collections import Counter
print(Counter(predict1))
#print(Counter(predict2))
#print(predict)

Counter({0: 170, 1: 30})


In [15]:
export_data=[{'id':i,'category':c} for i,c in enumerate(predict1)]


In [16]:
import csv
header=['id', 'category']
csv_file = "submission10.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        writer.writeheader()
        for data in export_data:
            writer.writerow(data)
except IOError:
    print("I/O error") 